In [ ]:
pip install --ignore-installed --upgrade tensorflow-gpu

In [ ]:
import sys
!{sys.executable} -m pip install transformers

In [ ]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

In [3]:
import pandas as pd
import os

In [4]:
df_comments = pd.read_pickle('07142022_clean_data.pkl')

In [5]:
df_comments.video_link.unique()

array(['Bed Intruder', 'Sweet Brown', 'Annoying Orange', 'Justice League',
       'World Star'], dtype=object)

In [28]:
df = df_comments[df_comments.video_link == 'World Star']

In [30]:
print(df.shape)
df = df.reset_index(drop=True)

(478, 22)


In [31]:
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer

# Create class for data preparation
class SimpleDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts

    def __len__(self):
        return len(self.tokenized_texts["input_ids"])

    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}

In [32]:
pred_texts = df['clean_comments'].astype('str').tolist()

In [33]:
# load tokenizer and model, create trainer
model_name = "j-hartmann/emotion-english-distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
trainer = Trainer(model=model)

loading configuration file https://huggingface.co/j-hartmann/emotion-english-distilroberta-base/resolve/main/config.json from cache at /N/u/abbajpai/Carbonate/.cache/huggingface/transformers/cf0d76ef122b326527bd6a56a127784dd513117bdfde0df5b011c9751bce8fd6.512111953428ded6703782e47e2f67b1c438267f7f4a7c0f19bc1850651a93b4
Model config RobertaConfig {
  "_name_or_path": "j-hartmann/emotion-english-distilroberta-base",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "anger",
    "1": "disgust",
    "2": "fear",
    "3": "joy",
    "4": "neutral",
    "5": "sadness",
    "6": "surprise"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "anger": 0,
    "disgust": 1,
    "fear": 2,
    "joy": 3,
    "n

In [34]:
# Tokenize texts and create prediction data set
tokenized_texts = tokenizer(pred_texts,truncation=True,padding=True)
pred_dataset = SimpleDataset(tokenized_texts)

In [35]:
# Run predictions
predictions = trainer.predict(pred_dataset)

***** Running Prediction *****
  Num examples = 478
  Batch size = 8


In [36]:
# Transform predictions to labels
preds = predictions.predictions.argmax(-1)
labels = pd.Series(preds).map(model.config.id2label)
scores = (np.exp(predictions[0])/np.exp(predictions[0]).sum(-1,keepdims=True)).max(1)

In [37]:
# scores raw
temp = (np.exp(predictions[0])/np.exp(predictions[0]).sum(-1,keepdims=True))

In [ ]:
df['anger']= None
df['disgust']= None
df['fear']= None
df['joy']= None
df['neutral']= None
df['sadness']= None
df['surprise']= None

for index, row in df.iterrows():
    df.loc[index, 'anger']= temp[index][0]
    df.loc[index, 'disgust']= temp[index][1]
    df.loc[index, 'fear']= temp[index][2]
    df.loc[index, 'joy']= temp[index][3]
    df.loc[index, 'neutral']= temp[index][4]
    df.loc[index, 'sadness']= temp[index][5]
    df.loc[index, 'surprise']= temp[index][6]
    print(index)

In [39]:
df.to_pickle('World_Star_Comments_Key_Emotions.pkl')